### Parsing

In [36]:
test_input = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
"""

with open("inputs/d18") as f:
    input = f.read()

In [54]:
def parse_input(input):
    plan = [
        dict(zip(["direction", "meters", "color"], line.split()))
        for line in input.strip().splitlines()
    ]

    for step in plan:
        step["meters"] = int(step["meters"])
        step["color"] = step["color"].strip("()")

    return plan

In [216]:
test_plan = parse_input(test_input)
plan = parse_input(input)

### Part 1: Flood Fill Solution and Polygon Visualization

In [55]:
def build_boundary(plan):
    boundary = set()
    min_row, min_col, max_row, max_col, row, col = 0, 0, 0, 0, 0, 0
    direction_moves = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}

    for step in plan:
        move = direction_moves[step["direction"]]
        for _ in range(step["meters"]):
            row = row + move[0]
            col = col + move[1]
            boundary.add((row, col))
        min_row = min(min_row, row)
        min_col = min(min_col, col)
        max_row = max(max_row, row)
        max_col = max(max_col, col)

    return {"boundary": boundary, "min_row": min_row, "max_row": max_row, "min_col": min_col, "max_col": max_col}

In [217]:
test_boundary = build_boundary(test_plan)
assert len(test_boundary["boundary"]) == 38

In [218]:
boundary = build_boundary(plan)
assert len(boundary["boundary"]) == 3650

In [100]:
def visualize(boundary: dict, inner: set):
    for row in range(boundary["min_row"], boundary["max_row"] + 1):
        for col in range(boundary["min_col"], boundary["max_col"] + 1):
            if (row, col) in boundary["boundary"]:
                print("#", end="")
            elif (row, col) in inner:
                print("x", end="")
            else:
                print(".", end="")
        print()

In [96]:
visualize(test_boundary, set())

#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######


In [109]:
def flood_fill(boundary: dict):
    def find_start():
        """Find left-most point we can start flood fill from"""
        for row in range(boundary["min_row"], boundary["max_row"] + 1):
            if (row, 0) in boundary["boundary"]:
                return (row, 0)

    (row, col) = find_start()
    row += 1
    col += 1

    neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    frontier = [(row, col)]
    found = set()
    while frontier:
        (row, col) = frontier.pop()
        for (delta_row, delta_col) in neighbours:
            neighbour = (row + delta_row, col + delta_col)
            if neighbour[0] >= boundary["min_row"] and neighbour[0] <= boundary["max_row"] and \
                neighbour[1] >= boundary["min_col"] and neighbour[1] <= boundary["max_col"] and \
                neighbour not in boundary["boundary"] and neighbour not in found:

                found.add(neighbour)
                frontier.append(neighbour)
    return found


In [222]:
test_inner = flood_fill(test_boundary)
visualize(test_boundary, test_inner)

#######
#xxxxx#
###xxx#
..#xxx#
..#xxx#
###x###
#xxx#..
##xx###
.#xxxx#
.######


In [221]:
# inner = flood_fill(boundary)
# visualize(boundary, inner)

In [107]:
def p1(plan):
    boundary = build_boundary(plan)
    inner = flood_fill(boundary)
    return len(boundary["boundary"]) + len(inner)

In [210]:
assert p1(test_plan) == 62

In [220]:
%%time

assert p1(plan) == 56923

CPU times: user 58 ms, sys: 0 ns, total: 58 ms
Wall time: 57.7 ms


### Part 2: Solution with Shoelace Formula

Since we cannot realistically build such a huge set and do flood fill on it, 
we instead use  [Shoelace formula](https://en.wikipedia.org/wiki/Shoelace_formula) to compute the area
of the polygon.

We are using trapezioid variation of the formula, and it seems that we already include 
half the boundary in the shoelace area, so we only use half of the boundary_length to arrive at the final
answer in `p2`.

In [223]:
import itertools

In [124]:
def reinterpret(plan):
    directions = ["R", "D", "L", "U"]
    steps = []

    for step in plan:
        digits = step["color"].removeprefix("#")
        meters = int(digits[0:5], 16)
        direction = directions[int(digits[5])]

        steps.append({"direction": direction, "meters": meters})

    return steps

In [166]:
def vertices_and_boundary_length(real_plan):
    vertices = []
    boundary_length = 0
    min_x, min_y, max_x, max_y, row, col = 0, 0, 0, 0, 0, 0
    direction_moves = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}

    for step in real_plan:
        move = direction_moves[step["direction"]]
        boundary_length += step["meters"]
        row = row + move[0] * step["meters"]
        col = col + move[1] * step["meters"]
        # here we interpret them as (x, y) coords, to apply shoelace formula
        vertices.append((col, row))

        min_x = min(min_x, col)
        min_y = min(min_y, row)
        max_x = max(max_x, col)
        max_y = max(max_y, row)


    return (vertices, boundary_length)

In [164]:
def shoelace_area(vertices):
    area = 0
    for ((x1, y1), (x2, y2)) in itertools.pairwise(vertices):
        area += (y1 + y2) * (x1 - x2)
    return 0.5 * area

In [219]:
def p2(plan, should_reinterpret=True):
    real_plan = reinterpret(plan) if should_reinterpret else plan
    (vertices, boundary_length) = vertices_and_boundary_length(real_plan)
    return int(shoelace_area(vertices) + boundary_length / 2 + 1)

In [208]:
assert p2(test_plan) == 952408144115
assert p2(test_plan, False) == 62
assert p2(plan, False) == 56923

In [214]:
%%time

assert p2(plan) == 66296566363189

CPU times: user 1.03 ms, sys: 0 ns, total: 1.03 ms
Wall time: 1.03 ms
